In [1]:
import pickle

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
import wikipedia
import networkx as nx
import urllib.request
from bs4 import BeautifulSoup
import re
import csv

contents = urllib.request.urlopen("https://en.wikipedia.org/wiki/Historical_federal_electoral_districts_of_Canada").read()

section = wikipedia.WikipediaPage('Historical federal electoral districts of Canada')
years = section.links

regex = re.compile("(List of Canadian )(federal )?(electoral districts)")

years = [s for s in years if regex.match(s)]
#years = years[:3]
years = years[12:]
#years = years[-5:]
for year in years:
    print(year)
#years = years[-2:]

# Construct a dict for the riding names for the entire set of ridings
riding_dict = {}

class RidingObject:
    def __init__(self, name):
        self.name = name
        self.eras = []
        self.elections = []

class Era:
    def __init__(self, start, end, predecessors, successors):
        self.start = start
        self.end = end
        self.predecessors = predecessors
        self.successors = successors

    def add_dates(self, start, end):
        self.start = start
        self.end = end

    def add_predecessors (self, predecessors):
        self.predecessors = predecessors

    def add_successors(self, successors):
        self.successors = successors

List of Canadian electoral districts 1947–1952
List of Canadian electoral districts 1952–1966
List of Canadian electoral districts 1966–1976
List of Canadian electoral districts 1976–1987
List of Canadian electoral districts 1987–1996
List of Canadian electoral districts 1996–2003
List of Canadian electoral districts 2003–2013
List of Canadian federal electoral districts


In [3]:
# Go through and obtain the riding names for every year
for i in range(len(years)):#range(4): #range(len(years)):
    year = years[i]
    print(year)
    section = wikipedia.WikipediaPage(year)
    html = section.html()

    soup = BeautifulSoup(html, 'html.parser')

    # GET ELECTORAL DISTRICT NAMES
    lists = soup.find_all("ul")
    lists.pop(0)
    content = lists[0]

    def getArticleTitles(ul_list):
        content = ul_list.findChildren("li")
        return [getArticleTitle(article) for article in content]

    def getArticleTitle(list_element):
        return list_element.findChildren("a" , recursive=False)[0].get("title")
    
    if year == 'List of Canadian federal electoral districts':
        lists = lists[22:35]

    ridings_by_province = [getArticleTitles(list_element) for list_element in lists]

    num_districts = 0
    for district_list in ridings_by_province:
        for riding in district_list:
            riding_dict[riding] = RidingObject(riding)
            num_districts += 1
    
    print("Year: ", year)
    print("Number of districts: ", num_districts)
    #break

List of Canadian electoral districts 1947–1952
Year:  List of Canadian electoral districts 1947–1952
Number of districts:  260
List of Canadian electoral districts 1952–1966
Year:  List of Canadian electoral districts 1952–1966
Number of districts:  264
List of Canadian electoral districts 1966–1976
Year:  List of Canadian electoral districts 1966–1976
Number of districts:  263
List of Canadian electoral districts 1976–1987
Year:  List of Canadian electoral districts 1976–1987
Number of districts:  281
List of Canadian electoral districts 1987–1996
Year:  List of Canadian electoral districts 1987–1996
Number of districts:  295
List of Canadian electoral districts 1996–2003
Year:  List of Canadian electoral districts 1996–2003
Number of districts:  301
List of Canadian electoral districts 2003–2013
Year:  List of Canadian electoral districts 2003–2013
Number of districts:  308
List of Canadian federal electoral districts
Year:  List of Canadian federal electoral districts
Number of dist

In [4]:
print(list(riding_dict.keys()))

['Bonavista—Twillingate', 'Burin—Burgeo', 'Grand Falls—White Bay', "Humber—St. George's", "St. John's East", "St. John's West", 'Trinity—Conception', 'Annapolis—Kings', 'Antigonish—Guysborough', 'Cape Breton North and Victoria', 'Cape Breton South (federal electoral district)', 'Colchester—Hants', 'Cumberland (electoral district)', 'Digby—Yarmouth', 'Halifax (electoral district)', 'Inverness—Richmond', 'Lunenburg (electoral district)', 'Pictou (electoral district)', 'Queens—Shelburne', "King's (Prince Edward Island electoral district)", 'Prince (electoral district)', "Queen's (Prince Edward Island electoral district)", 'Charlotte (electoral district)', 'Gloucester (electoral district)', 'Kent (New Brunswick electoral district)', 'Northumberland (New Brunswick electoral district)', 'Restigouche—Madawaska', 'Royal (electoral district)', 'St. John—Albert', 'Victoria—Carleton', 'Westmorland (electoral district)', 'York—Sunbury', 'Argenteuil—Deux-Montagnes', 'Beauce (electoral district)', '

In [5]:
# LOAD CSV DISTRICT NAMES
import pandas as pd

filename = "./ParlinfoRidings.csv"

data = pd.read_csv(filename, sep=',', delimiter=None, header='infer')
parl_names = data["Name"]
print(parl_names)
print(list(parl_names))

0               Cariboo District
1       New Westminster District
2               Vancouver Island
3              Victoria District
4                  Yale District
                  ...           
1555                 York Centre
1556          York South--Weston
1557                York--Simcoe
1558           Yorkton--Melville
1559                       Yukon
Name: Name, Length: 1560, dtype: object
['Cariboo District', 'New Westminster District', 'Vancouver Island', 'Victoria District', 'Yale District', 'Bruce South', 'Cornwall', 'Essex', 'Huron Centre', 'Huron North', 'Lambton', 'Lincoln', 'Muskoka', 'Niagara', 'Stormont', 'Chambly', 'Brant North', 'Cariboo', 'Iberville', "King's County", 'Laprairie', 'Montreal Centre', 'Montreal East', 'Montreal West', 'Napierville', 'Ottawa (County of)', 'Prince County', "Queen's", "Queen's County", 'Queens', 'Saint Maurice', 'Shelburne', "St. John's", 'Sunbury', 'Three Rivers', 'Verchères', 'Wentworth North', 'Yale', 'Haldimand', 'Monck', 'Addingt

In [6]:
import re

# Construct a dict for the riding names for the entire set of ridings
translate_dict = {}

electoral_districts =  list(riding_dict.keys()) 
num_districts = len(electoral_districts)
parl_list = list(parl_names)

for i in range(num_districts):
    article_title = electoral_districts[i]
    if article_title is not None:
        #article_title = "Digby and Annapolis"
        dict_title = article_title

        #print(article_title, " with index ", i, "/", num_districts)

        # Remove (electoral district)
        article_title = re.sub("\s?(\(.*[electoral|eletoral] district\))", "", article_title)
        #print(article_title)

        # Replace "—" with "--"
        article_title = re.sub("(Essex-)", "Essex—", article_title)
        article_title = re.sub("—", "--", article_title)
        #print(article_title)

        # Manual fixes (mainly french nams)
        article_title = re.sub("(Grand-Falls)", "Grand Falls", article_title)
        article_title = re.sub("(Temiscouata)", "Témiscouata", article_title)
        article_title = re.sub("(Beausejour)", "Beauséjour", article_title)

        article_title = re.sub("(Southwest Nova)", "South West Nova", article_title)
        article_title = re.sub("(Winnipeg St. James)", "Winnipeg--St. James", article_title)
        article_title = re.sub("(Orleans)", "Orléans", article_title)
        article_title = re.sub("(Rimouski--Neigette-et-La-Mitis)", 'Rimouski-Neigette-et-La Mitis', article_title)

        article_title = re.sub("(Hamilton--Mountain)", "Hamilton Mountain", article_title)
        article_title = re.sub("(London North-Centre)", "London North Centre", article_title)
        article_title = re.sub("(Vaudreuil-Soulanges)", "Vaudreuil--Soulanges", article_title)

        article_title = re.sub("(Montmagny--L'Islet--Kamouraska--Rivière-du-Loup)", "Montmagny--L’Islet--Kamouraska--Rivière-du-Loup", article_title)
        article_title = re.sub("(Beauport--Côte-de-Beaupré--Île d’Orléans--Charlevoix)", "Beauport--Côte-de-Beaupré--Île D’Orléans--Charlevoix", article_title)
        article_title = re.sub("(Ville-Marie--Le Sud-Ouest--Île-des-Sœurs)", "Ville-Marie--Le Sud-Ouest--Île-des-Soeurs", article_title)

        article_title = re.sub("(Joliette--L'Assomption--Montcalm)", "Joliette--l'Assomption--Montcalm", article_title)
        article_title = re.sub("(Montmagny--L'Islet)", "Montmagny--l'Islet", article_title)
        article_title = re.sub("(Berthier--Maskinongé--delanaudière)", "Berthier--Maskinongé--de Lanaudière", article_title)
        article_title = re.sub("(Jacques Cartier--Lasalle)", "Jacques-Cartier--Lasalle", article_title)
        article_title = re.sub("(Bellechasse--Etchemins--Montmagny--l'Islet)", "Bellechasse--Etchemins--Montmagny--L'Islet", article_title)


        if (article_title not in parl_list):
            print("#########################################################")
            print(article_title, " not in parl names")
        
        translate_dict[dict_title] = article_title
        
print(list(translate_dict.items()))

[('Bonavista—Twillingate', 'Bonavista--Twillingate'), ('Burin—Burgeo', 'Burin--Burgeo'), ('Grand Falls—White Bay', 'Grand Falls--White Bay'), ("Humber—St. George's", "Humber--St. George's"), ("St. John's East", "St. John's East"), ("St. John's West", "St. John's West"), ('Trinity—Conception', 'Trinity--Conception'), ('Annapolis—Kings', 'Annapolis--Kings'), ('Antigonish—Guysborough', 'Antigonish--Guysborough'), ('Cape Breton North and Victoria', 'Cape Breton North and Victoria'), ('Cape Breton South (federal electoral district)', 'Cape Breton South'), ('Colchester—Hants', 'Colchester--Hants'), ('Cumberland (electoral district)', 'Cumberland'), ('Digby—Yarmouth', 'Digby--Yarmouth'), ('Halifax (electoral district)', 'Halifax'), ('Inverness—Richmond', 'Inverness--Richmond'), ('Lunenburg (electoral district)', 'Lunenburg'), ('Pictou (electoral district)', 'Pictou'), ('Queens—Shelburne', 'Queens--Shelburne'), ("King's (Prince Edward Island electoral district)", "King's"), ('Prince (electoral

In [7]:
# SAVE MAP
map_filename = './data_parl_map.pickle'

#pickle.dump(translate_dict, open("data_parl_map.p", "wb"))

In [8]:
with open(map_filename, 'wb') as handle:
    pickle.dump(translate_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:
with open(map_filename, 'rb') as handle:
    pickle_map = pickle.load(handle)
print(list(pickle_map.items()))

[('Bonavista—Twillingate', 'Bonavista--Twillingate'), ('Burin—Burgeo', 'Burin--Burgeo'), ('Grand Falls—White Bay', 'Grand Falls--White Bay'), ("Humber—St. George's", "Humber--St. George's"), ("St. John's East", "St. John's East"), ("St. John's West", "St. John's West"), ('Trinity—Conception', 'Trinity--Conception'), ('Annapolis—Kings', 'Annapolis--Kings'), ('Antigonish—Guysborough', 'Antigonish--Guysborough'), ('Cape Breton North and Victoria', 'Cape Breton North and Victoria'), ('Cape Breton South (federal electoral district)', 'Cape Breton South'), ('Colchester—Hants', 'Colchester--Hants'), ('Cumberland (electoral district)', 'Cumberland'), ('Digby—Yarmouth', 'Digby--Yarmouth'), ('Halifax (electoral district)', 'Halifax'), ('Inverness—Richmond', 'Inverness--Richmond'), ('Lunenburg (electoral district)', 'Lunenburg'), ('Pictou (electoral district)', 'Pictou'), ('Queens—Shelburne', 'Queens--Shelburne'), ("King's (Prince Edward Island electoral district)", "King's"), ('Prince (electoral